# 清览题库 同步器

In [1]:
import requests
import time

headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "zh-CN,zh;q=0.9,en;q=0.8",
    "authorization": "Bearer",
    "sec-ch-ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin"
  }

## 清览题库书本自动刷题脚本

In [20]:
type_list = ["单选题", "多选题", "判断题", "填空题", "简答题", "编程题"]


def post_answer(id):
    # 获取单道题目
    question_info = requests.get(f'https://www.qingline.net/api/question/{id}',
                                 headers=headers).json()

    time.sleep(5)

    # 页面信息
    page_info = requests.get(
        f'https://www.qingline.net/api/question/page-info/{id}', headers=headers).json()

    answer = []
    answer_content = []
    question_type = question_info['data']['type']
    for i in question_info['data']['right_answers']:
        if question_type in [3, 4, 5, 6]:  # 判断题、填空题、简答题、编程题
            answer.append(i['content'])
        else:
            answer.append(i['id'])

        answer_content.append(i['content'])

    # 提交题目
    post_back = requests.post(f'https://www.qingline.net/api/student/question/answer/{id}',
                              headers=headers,
                              json={"answer": answer}).json()

    return page_info['data']['next'], question_info['data']['stem'], answer_content, post_back['data']['is_right']

In [21]:
id = 48204
fp = open('answer.txt', 'w', encoding='utf-8')
while True:
    id, stem, answer_content, is_right = post_answer(id)
    print(stem)
    print(answer_content)
    print('提交成功' if is_right else '提交失败')
    print('----------------------')
    fp.write(stem + '\n')
    fp.write(str(answer_content) + '\n\n')
    fp.flush()
    if id == None:
        break
    
fp.close()

<p>Jupyter notebook不具备的功能是（）</p>
['<p>Jupyter notebook可以将文件分享给他人</p>']
提交成功
----------------------
在Jupyter notebook的命令模式下，要查看所有快捷键应该按下的快捷键是  （   ）
['H']
提交成功
----------------------
<p>在Jupyter notebook的cell中安装包语句正确的是                       （ ）</p>
['<p>!pip install 包名</p>']
提交成功
----------------------
<p>Numpy提供了两种基本对象，一种是ndarray，另一种是（   ）</p>
['<p>func</p>']
提交成功
----------------------
<p>创建一个3*3的数组，下列代码中错误的是（   ）</p>
['<p>np.random.random([3,3,3])</p>']
提交成功
----------------------
<p>Numpy中统计数组元素个数的方法是（    ）</p>
['<p>size</p>']
提交成功
----------------------
利用下面哪个可视化绘图可以发现数据的异常点。   （   ）
['盒图']
提交成功
----------------------
以下关于缺失值检测的说法中，正确的是（   ）
['Pandas库中的interpolate模块包含了多种插值方法']
提交成功
----------------------
Matplotlib中的哪个包提供了一批操作和绘图函数（ ）
['pyplot']
提交成功
----------------------
以下关于绘图标准流程说法错误的是（   ）
['添加图例可以在绘制图形之前']
提交成功
----------------------
下列参数中调整后显示中文的（     ）
['font.sans-serif']
提交成功
----------------------
以下说法错误的是（    ）
['散点图无法反映特征间的统计关系。']
提交成功
----------------------
列表是不可变对象，支持

KeyError: 'data'

## 清览题库自动答题脚本

- 获取所有作业

https://www.qingline.net/api/student/homework?page=1&page_number=10

- 获取作业信息

https://www.qingline.net/api/student/homework/short-info/1729407

- 获取作业题目数据

https://www.qingline.net/api/student/question/1729407?page=1&pageSize=100

- 获取作业保存的答案

https://www.qingline.net/api/student/homework/basic-answer/1729407

- 保存一道题目 POST

https://www.qingline.net/api/student/homework/1729407/fake-save-answer

Payload:
[{"question_id":48213,"answer":[123689]}]

In [2]:
def get_answer(id):
    # 获取题目的答案
    question_info = requests.get(f'https://www.qingline.net/api/question/{id}',
                                 headers=headers).json()
    answer = []
    question_type = question_info['data']['type']
    for i in question_info['data']['right_answers']:
        if question_type in [3, 4, 5, 6]:  # 判断题、填空题、简答题、编程题
            answer.append(i['content'])
        else:
            answer.append(i['id'])
    return answer


def get_undo_homework_list():
    # 获取未完成作业清单
    homework_list = requests.get(
        'https://www.qingline.net/api/student/homework?page=1&page_number=10', headers=headers).json()
    works = []
    for i in homework_list['data']['items']:
        if i['status'] == 0:
            works.append((i['id'], i['homework']['title']))
    return works


def get_homework_items(id):
    # 获取作业的题目
    homework_items = requests.get(
        f'https://www.qingline.net/api/student/question/{id}?page=1&pageSize=100', headers=headers).json()
    ids = [i['id'] for i in homework_items['data']['items']]
    return ids


def post_answers(id, qid, answer):
    # 暂存作业的提交答案
    post_back = requests.post(f'https://www.qingline.net/api/student/homework/{id}/fake-save-answer',
                              headers=headers,
                              json=[{"question_id": qid, "answer": answer}]).json()
    return post_back

In [3]:
for i in get_undo_homework_list():
    print(f"定位到作业ID:{i[0]}({i[1]})")
    items = get_homework_items(i[0])
    print(f"作业ID:{i[0]}({i[1]})共有{len(items)}道题目")
    for j in items:
        print(f"定位到题目ID:{j}，准备开始答题...")
        answer = get_answer(j)
        print(f"\t题目ID:{j}的答案为{answer}")
        time.sleep(2)
        msg = post_answers(i[0], j, answer)['data']
        print(f"\t题目ID:{j}提交，服务器返回：{msg}")
        time.sleep(3)
    print(f"作业ID:{i[0]}({i[1]})已完成，请自己提交。\n\n")

定位到作业ID:1729431(时间序列数据分析)
作业ID:1729431(时间序列数据分析)共有17道题目
定位到题目ID:48263，准备开始答题...
	题目ID:48263的答案为['date_range']
	题目ID:48263提交，服务器返回：操作成功
定位到题目ID:48264，准备开始答题...
	题目ID:48264的答案为['now()']
	题目ID:48264提交，服务器返回：操作成功
定位到题目ID:48265，准备开始答题...
	题目ID:48265的答案为['时间类型']
	题目ID:48265提交，服务器返回：操作成功
定位到题目ID:48266，准备开始答题...
	题目ID:48266的答案为['下采样']
	题目ID:48266提交，服务器返回：操作成功
定位到题目ID:48295，准备开始答题...
	题目ID:48295的答案为['在多个时间点观测或测量的数据形成了时间序列。时间序列数据是一种重要的结构化数据类型，如金融、经济、生态学、神经科学和物理学等领域。']
	题目ID:48295提交，服务器返回：操作成功
定位到题目ID:48296，准备开始答题...
	题目ID:48296的答案为['用Python来进行平稳性检验主要有时序图检验、自相关图检验以及构造统计量进行检验3种方法。\n（1）时序图\n时序图就是普通的时间序列图，即以时间为横轴，观察值为纵轴进行检验。利用时序图可以粗略观察序列的平稳性。\n（2）自相关图检验\n平稳序列通常具有短期相关性，即随着延迟期数k的增加，平稳序列的自相关系数会很快地衰减向零，而非平稳序列的自相关系数的衰减速度会比较慢。画自相关图用到的是statsmodels中的plot_acf方法。自相关图中横轴表示延迟期数，纵轴表示自相关系数。\n（3）构造统计量\n利用绘图判断序列的平稳性比较直观，但不够精确，ADF（Augmented Dickey-Fuller）法直接通过假设检验的方式来验证平稳性。ADF的原假设（H0）和备择假设（H1）如下：\nH0：具有单位根，属于非平稳序列；\nH1：没有单位根，属于平稳序列。\nPython中可以使用statsmodels中的adfuller方法进行ADF检验，直接输入数据，即可返回7个数值。其中的第一个返回值adf就是ADF方法的检验结果，